In [1]:
import numpy as np
import pandas as pd
import pickle
import mlflow
import sklearn
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient

import sklearn
from sklearn.metrics import r2_score,mean_squared_error

In [2]:
X_train = pd.read_parquet("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\X_train_df.parquet.gzip")
X_test = pd.read_parquet("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\X_test_df.parquet")
y_train = pd.read_csv("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\y_train_df.csv")
y_test = pd.read_csv("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\y_test_df.csv")


In [3]:
X_train.drop(columns=['Unnamed: 0'],inplace=True)
X_train.drop(columns=['Cpu'],inplace=True)
X_train.drop(columns=['Gpu'],inplace=True)
X_train.drop(columns=['Cpu Name'],inplace=True)
y_train.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
X_test.drop(columns=['Unnamed: 0'],inplace=True)
X_test.drop(columns=['Cpu'],inplace=True)
X_test.drop(columns=['Gpu'],inplace=True)
X_test.drop(columns=['Cpu Name'],inplace=True)
y_test.drop(columns=['Unnamed: 0'],inplace=True)

In [6]:
MLFLOW_TRACKING_URI = "sqlite:///demo.db"
prod_stage="Production"
staging_stage = "Staging"
model_name ="laptop-price-demo"
mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [7]:
# try -catch exception clause
try:
    model = mlflow.pyfunc.load_model(f"models:/{model_name}/{prod_stage}")
except mlflow.MlflowException:
    print("There is no such model or model version in MLFlow production registry")
else:
    print("The model is present we can use it")


The model is present we can use it


In [8]:
X_train.head(1)

,Company,TypeName,Ram,Weight,Touchscreen,Ips,ppi,Cpu brand,HDD,SSD,Gpu brand,os
1259,MSI,Gaming,8,2.4,0,0,141.211998,Intel Core i7,1000,128,Nvidia,Windows


In [53]:
column_transformer = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,10,11])
],remainder='passthrough')
X_train_tranformed = column_transformer.fit_transform(X_train)
X_val_tranformed = column_transformer.transform(X_test)

In [56]:
X_train_tranformed.shape

(1041, 38)

In [44]:
X_val_tranformed.shape

(261, 38)

In [45]:
X_train_tranformed.shape

(1041, 38)

In [ ]:
y_pred = model.predict(X_test)

In [15]:
r2_score(y_test, y_pred)

0.8836828236626981

In [ ]:
"./mlruns/2/128731dce22e4a06bed20def94cf81db/artifacts/model/model.pkl"

In [9]:
def test_model(model_name, stage, X_val, y_val):
    model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")
    y_pred = model.predict(X_val)
    return r2_score(y_val, y_pred), model

In [10]:
r2, model = test_model(model_name, "Production", X_test, y_test)

In [11]:
print(f"R2 score : {r2}")

R2 score : 0.8836828236626981


In [12]:
def model_evaluator(
    X_train : pd.DataFrame,
    X_val: pd.DataFrame,
    y_val : pd.DataFrame,
):
    """Calculate the the R2 of the model """

    # column_transformer = ColumnTransformer(transformers=[
    #         ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,10,11])
    #     ],remainder='passthrough')
    # X_train_tranformed = column_transformer.fit_transform(X_train)
    # X_val_tranformed = column_transformer.transform(X_val)

    model_name = "laptop-price-demo"
    prod_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{prod_stage}")
    prod_model_r2_score, model = test_model(model_name=model_name, stage="Production", X_val=X_val, y_val=y_val)
    return prod_model_r2_score, model
    # try:
    #     prod_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{prod_stage}")

    # except mlflow.MlflowException:
    #     print("There is no such model or model version in MLFlow production registry")
    #     # registering the fresh new model from staging to production stage 
    #     client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
    #     latest_versions = client.get_latest_versions(name=model_name)

    #     for version in latest_versions:
    #         model_version = version.version
        
    #     new_stage = "Production"
    #     client.transition_model_version_stage(
    #         name=model_name,
    #         version=model_version,
    #         stage=new_stage,
    #         archive_existing_versions=True
    #     )
        
    #     # update the mlflow client
    #     date = datetime.today().date()
    #     client.update_model_version(
    #         name=model_name,
    #         version=model_version,
    #         description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
    #     )
    #     print(f"The new model version of {model_name} has been placed in production stage")
    #     # prod_model_r2_score, model = test_model(model_name=model_name, stage="Production", X_val=X_val_tranformed, y_val=y_val)
    #     #return prod_model_r2_score, model
        
    # else:
    #     print("The model is present we can use it")
    #     # production model
    #     prod_r2_score, model = test_model(name=model_name, stage="Production", X_val=X_val_tranformed, y_val=y_val)
    #     # staging Model
    #     staging_r2_score, model = test_model(name=model_name, stage="Staging", X_val=X_val_tranformed, y_val=y_val)
    #     if staging_r2_score < prod_r2_score:
    #         # continue as normal
    #         print("The production model is performing good. Leave it!")
    #         pass
    #     else:
    #         # registering the fresh new model into production
    #         client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
    #         latest_versions = client.get_latest_versions(name=model_name)

    #         for version in latest_versions:
    #             model_version = version.version
            
    #         new_stage = "Production"
    #         client.transition_model_version_stage(
    #             name=model_name,
    #             version=model_version,
    #             stage=new_stage,
    #             archive_existing_versions=True
    #         )
            
    #         # update the mlflow client
    #         date = datetime.today().date()
    #         client.update_model_version(
    #             name=model_name,
    #             version=model_version,
    #             description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
    #         )
    #     # prod_model_r2_score, model = test_model(name=model_name, stage="Production", X_val=X_val_tranformed, y_val=y_val)
    #     #return prod_model_r2_score, model
    # prod_model_r2_score, model = test_model(model_name=model_name, stage="Production", X_val=X_val_tranformed, y_val=y_val)
    # # return prod_model_r2_score, model

In [13]:
r2, model = model_evaluator(X_train, X_test, y_test)

In [14]:
print(f"Evaluation R2 score : {r2}")

Evaluation R2 score : 0.8836828236626981


In [62]:
X_test.head()

,Company,TypeName,Ram,Weight,Touchscreen,Ips,ppi,Cpu brand,HDD,SSD,Gpu brand,os
1233,MSI,Gaming,16,2.70,0,0,127.335675,Intel Core i7,1000,256,Nvidia,Windows
1030,HP,Notebook,4,1.64,0,0,111.935204,Intel Core i5,0,256,Intel,Windows
163,Lenovo,Gaming,16,2.50,0,1,141.211998,Intel Core i7,0,256,Nvidia,Windows
1012,HP,Notebook,4,1.48,0,0,157.350512,Intel Core i5,0,256,Intel,Windows
711,Dell,Notebook,8,1.64,0,0,157.350512,Intel Core i5,0,256,Nvidia,Others/No OS/Linux


In [34]:

    
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

In [35]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
model_name = "laptop-price-model"
stage = "Staging"

In [36]:
from mlflow.tracking import MlflowClient

In [37]:
client = MlflowClient()
# retrieve the top_n model runs and log the models to MLflow
experiment = client.get_experiment_by_name("hyper-optimization-experiment-1")

In [38]:
experiment.experiment_id

'2'

In [33]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")

OSError: No such file or directory: '.\mlruns\2\ff252e28434f45408894dcfb55f94b31\artifacts\model\.'